In [55]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import scipy
from time import time

In [28]:
bow=catalog.load('BOW_train')
vocab=catalog.load('UN_dictionary')
beta=catalog.load('Word_distribution')

2020-03-26 15:06:28,895 - kedro.io.data_catalog - INFO - Loading data from `BOW_train` (ScipySparseMatrix)...
2020-03-26 15:06:29,102 - kedro.io.data_catalog - INFO - Loading data from `UN_dictionary` (DictionaryDataSet)...
2020-03-26 15:06:29,104 - gensim.utils - INFO - loading Dictionary object from data/05_model_input/UN_dictionary.dict
2020-03-26 15:06:29,119 - gensim.utils - INFO - loaded data/05_model_input/UN_dictionary.dict
2020-03-26 15:06:29,126 - kedro.io.data_catalog - INFO - Loading data from `Word_distribution` (NumpyArray)...


We want to test the time it takes to calculate TC with two different technics : 

the first one is the one implemented on the DETM paper, we use it as a benchmark : we know it provides the good results, but it takes a very long time to fit.

The second one is implemented by us, it is based on the direct Bag Of Words to calculate occurences & co occurences. We want to see if it provides the same results on a shorter fitting time

### The goal is to compare 2 way of calculating Topic Coherence

##### 1. The first one is based on DETM paper implementation

In [51]:
def split_bow_2(bow_in, n_docs):
    indices = [[w for w in bow_in[doc,:].indices] for doc in range(n_docs)]
    indices_arr=[np.array(element) for element in indices]
    counts = [[c for c in bow_in[doc,:].data] for doc in range(n_docs)]
    counts_arr=[np.array(element) for element in counts]

    return np.array(indices_arr), np.array(counts_arr)

tokens,counts=split_bow_2(bow,bow.shape[0])

In [52]:
def get_document_frequency(data, wi, wj=None):
    if wj is None:
        D_wi = 0
        for l in range(len(data)):
            #doc = data[l].squeeze(0)
            #if len(doc) == 1: 
            #    continue
                #doc = [doc.squeeze()]
            #else:
            #    doc = doc.squeeze()
            if wi in data[l]:
                D_wi += 1
        return D_wi
    D_wj = 0
    D_wi_wj = 0
    for l in range(len(data)):
        #doc = data[l].squeeze(0)
        #if len(doc) == 1: 
        #    doc = [doc.squeeze()]
        #else:
        #    doc = doc.squeeze()
        if wj in data[l]:
            D_wj += 1
            if wi in data[l]:
                D_wi_wj += 1
    return D_wj, D_wi_wj 


def get_topic_coherence(data, beta, num_topics, num_coherence):

    D = len(data) ## number of docs...data is list of documents
    TC = []
   
    for k in range(num_topics):
        print('\tDone {}/{}'.format(k,num_topics))
        top_10 = list(beta[k].argsort()[-num_coherence:][::-1])
        #top_words = [vocab[a] for a in top_10]

            
        
        TC_k = 0
        counter = 0
        for i, word in enumerate(top_10):
            # get D(w_i)
            D_wi = get_document_frequency(data, word)
            p_wi=D_wi/D
            j = i + 1
            tmp = 0
            while j < len(top_10) and j > i:
                # get D(w_j) and D(w_i, w_j)
                D_wj, D_wi_wj = get_document_frequency(data, word, top_10[j])
                p_wj=D_wj/D
                p_wi_wj=D_wi_wj/D

                if D_wi_wj == 0 :
                    tc_pairwise=-1
                elif D_wi_wj==D_wi and D_wi_wj==D_wj : 
                    tc_pairwise=1
                # get f(w_i, w_j)
                else : 
                    tc_pairwise = np.log(p_wi_wj/(p_wi*p_wj))/-np.log(p_wi_wj)

                # update tmp: 

                tmp += tc_pairwise
                j += 1
                counter += 1
            # update TC_k
            TC_k += tmp 
        TC_k=TC_k/counter
        TC.append(TC_k)
    print(TC)    
    #TC = np.mean(TC) / counter
    return TC

In [67]:
def model_topic_coherence(data,beta,num_times,num_topics,num_coherence=6) : 
    
    tc=np.zeros((num_times,num_topics))
    
    times=[]
    for timestep in range(num_times): 
        t0=time()

        print('-'*100)
        print('Timestep {}/{}'.format(timestep,num_times))
        print('-'*100)
        print('\n')
        tc[timestep,:]=get_topic_coherence(data,beta[:,timestep,:],num_topics,num_coherence)
        
        fitting_time=time()-t0
        
        print('\nFitting time : {}s\n'.format(round(fitting_time,2)))
        
        times.append(fitting_time)
    total_time=round(np.sum(times),2)
    print('Total fitting time for {} timestep & {} number of words is : {}s'.format(num_times,num_coherence,total_time))
    return tc

In [68]:
test_number_of_times=10
test_number_of_words=10

print('Total number of documents : {}'.format(bow.shape[0]))
print('Total number of words : {}'.format(bow.shape[1]))
print('Test number of timestep : {}'.format(test_number_of_times))
print('Test number of words to calcualte co occurences : {}'.format(test_number_of_words))


Total number of documents : 6382
Total number of words : 10812
Test number of timestep : 10
Test number of words to calcualte co occurences : 10


In [69]:
test_1=model_topic_coherence(tokens,beta,test_number_of_times,2,test_number_of_words)

----------------------------------------------------------------------------------------------------
Timestep 0/10
----------------------------------------------------------------------------------------------------


	Done 0/2
	Done 1/2
[0.033995217221109944, 0.04215773577567638]

Fitting time : 5.02s

----------------------------------------------------------------------------------------------------
Timestep 1/10
----------------------------------------------------------------------------------------------------


	Done 0/2
	Done 1/2
[0.11902849862264936, 0.043468405456986756]

Fitting time : 5.2s

----------------------------------------------------------------------------------------------------
Timestep 2/10
----------------------------------------------------------------------------------------------------


	Done 0/2
	Done 1/2
[0.08711458931198925, 0.07574572358603056]

Fitting time : 4.96s

---------------------------------------------------------------------------------------

In [71]:
print('Final Topic Coherence : ')
print(test_1)

Final Topic Coherence : 
[[0.03399522 0.04215774]
 [0.1190285  0.04346841]
 [0.08711459 0.07574572]
 [0.06410443 0.07755732]
 [0.10661991 0.09863093]
 [0.09884464 0.11751429]
 [0.13676235 0.06035408]
 [0.12208613 0.10098748]
 [0.07004846 0.11633656]
 [0.14564323 0.06052879]]


##### 2. Based on BOW representation of text

In [ ]:
def get_doc_freq(bow,wi,wj=None): 
    if wj is None : 
        return bow[:,wi].sum(axis=0)
    new=bow[:,wi]+bow[:,wj]
    return bow[:,wj].sum(axis=0),new[new==2].shape[0]

In [75]:
def get_one_hot_bow(bow) :
    """This function takes in input a basic BOW such as CountVecotrizer BOWs and return a one-hot BOW. This is a BOW where 
       each element (i,j) of the matrix is either a 0 if the word j is not in document i, and 1 if word j is in doc j.
       This differs from original BOW as in these standard BOW, each element (i,j) of the matrix is either 0 if the word j 
       is in document i or n_occu where n_occu is an integer that represents the number of times the word j appears in document i"""
    t0=time()
    
    bow_new=np.zeros((bow.shape[0],bow.shape[1]))
    for idx in range(bow.shape[0]) : 
        for i in np.argwhere(bow[idx]) :
            bow_new[idx,i[1]]=1
    print('Fitting time is : {}s'.format(round(time()-t0,2)))
    return bow_new

In [76]:
bow_test=get_one_hot_bow(bow)

Fitting time is : 4.31s


In [77]:
def get_topic_coherence_2(data, beta, num_topics, num_coherence):

    D = len(data) ## number of docs...data is list of documents
    TC = []
   
    for k in range(num_topics):
        print('\tDone {}/{}'.format(k,num_topics))
        top_10 = list(beta[k].argsort()[-num_coherence:][::-1])
        #top_words = [vocab[a] for a in top_10]

            
        
        TC_k = 0
        counter = 0
        for i, word in enumerate(top_10):
            # get D(w_i)
            D_wi = get_doc_freq(data, word)
            p_wi=D_wi/D
            j = i + 1
            tmp = 0
            while j < len(top_10) and j > i:
                # get D(w_j) and D(w_i, w_j)
                D_wj, D_wi_wj = get_doc_freq(data, word, top_10[j])
                p_wj=D_wj/D
                p_wi_wj=D_wi_wj/D

                if D_wi_wj == 0 :
                    tc_pairwise=-1
                elif D_wi_wj==D_wi and D_wi_wj==D_wj : 
                    tc_pairwise=1
                # get f(w_i, w_j)
                else : 
                    tc_pairwise = np.log(p_wi_wj/(p_wi*p_wj))/-np.log(p_wi_wj)

                # update tmp: 

                tmp += tc_pairwise
                j += 1
                counter += 1
            # update TC_k
            TC_k += tmp 
        TC_k=TC_k/counter
        TC.append(TC_k)
    print(TC)    
    #TC = np.mean(TC) / counter
    return TC

In [78]:
def model_topic_coherence_2(data,beta,num_times,num_topics,num_coherence=6) : 
    

    tc=np.zeros((num_times,num_topics))
    
    times=[]
    for timestep in range(num_times): 
        t0=time()

        print('-'*100)
        print('Timestep {}/{}'.format(timestep,num_times))
        print('-'*100)
        print('\n')
        tc[timestep,:]=get_topic_coherence_2(data,beta[:,timestep,:],num_topics,num_coherence)
        
        fitting_time=time()-t0
        
        print('\nFitting time : {}s\n'.format(round(fitting_time,2)))
        
        times.append(fitting_time)
    total_time=round(np.sum(times),2)
    print('Total fitting time for {} timestep & {} number of words is : {}s'.format(num_times,num_coherence,total_time))
    return tc

In [81]:
test_2=model_topic_coherence_2(bow_test,beta,test_number_of_times,2,test_number_of_words)

----------------------------------------------------------------------------------------------------
Timestep 0/10
----------------------------------------------------------------------------------------------------


	Done 0/2
	Done 1/2
[0.033995217221109944, 0.04215773577567638]

Fitting time : 0.07s

----------------------------------------------------------------------------------------------------
Timestep 1/10
----------------------------------------------------------------------------------------------------


	Done 0/2
	Done 1/2
[0.11902849862264936, 0.043468405456986756]

Fitting time : 0.07s

----------------------------------------------------------------------------------------------------
Timestep 2/10
----------------------------------------------------------------------------------------------------


	Done 0/2
	Done 1/2
[0.08711458931198925, 0.07574572358603056]

Fitting time : 0.07s

--------------------------------------------------------------------------------------

In [82]:
print('Final Topic Coherence : ')
print(test_2)

Final Topic Coherence : 
[[0.03399522 0.04215774]
 [0.1190285  0.04346841]
 [0.08711459 0.07574572]
 [0.06410443 0.07755732]
 [0.10661991 0.09863093]
 [0.09884464 0.11751429]
 [0.13676235 0.06035408]
 [0.12208613 0.10098748]
 [0.07004846 0.11633656]
 [0.14564323 0.06052879]]


##### Check differences

In [87]:
print('TC with method 1 :')
print(test_1)
print('\n\tFitting time : 51.14s')
print('\n')
print('TC with method 2 :')
print(test_2)
print('\n\tFitting time : 4.31s for creating one-hot-BOW, 0.73s for fitting time')

TC with method 1 :
[[0.03399522 0.04215774]
 [0.1190285  0.04346841]
 [0.08711459 0.07574572]
 [0.06410443 0.07755732]
 [0.10661991 0.09863093]
 [0.09884464 0.11751429]
 [0.13676235 0.06035408]
 [0.12208613 0.10098748]
 [0.07004846 0.11633656]
 [0.14564323 0.06052879]]

	Fitting time : 51.14s


TC with method 2 :
[[0.03399522 0.04215774]
 [0.1190285  0.04346841]
 [0.08711459 0.07574572]
 [0.06410443 0.07755732]
 [0.10661991 0.09863093]
 [0.09884464 0.11751429]
 [0.13676235 0.06035408]
 [0.12208613 0.10098748]
 [0.07004846 0.11633656]
 [0.14564323 0.06052879]]

	Fitting time : 4.31s for creating one-hot-BOW, 0.73s for fitting time


We obtained the exact same results and divided the time of fitting by 10 !

#### An other way to benchmark : varying the number of words 

In [91]:
test_number_of_words=30
test_number_of_times=3 #We reduce this in order to get less total computation time

In [92]:
test_1_=model_topic_coherence(tokens,beta,test_number_of_times,2,test_number_of_words)

----------------------------------------------------------------------------------------------------
Timestep 0/3
----------------------------------------------------------------------------------------------------


	Done 0/2
	Done 1/2
[0.03952746930657039, 0.04122628159712735]

Fitting time : 43.04s

----------------------------------------------------------------------------------------------------
Timestep 1/3
----------------------------------------------------------------------------------------------------


	Done 0/2
	Done 1/2
[0.0527796056484553, 0.06979573092922156]

Fitting time : 42.04s

----------------------------------------------------------------------------------------------------
Timestep 2/3
----------------------------------------------------------------------------------------------------


	Done 0/2
	Done 1/2
[0.08259622498258, 0.07429565904991407]

Fitting time : 45.07s

Total fitting time for 3 timestep & 30 number of words is : 130.15s


In [94]:
print('Final Topic Coherence : ')
print(test_1_)

Final Topic Coherence : 
[[0.03952747 0.04122628]
 [0.05277961 0.06979573]
 [0.08259622 0.07429566]]


In [95]:
test_2_=model_topic_coherence_2(bow_test,beta,test_number_of_times,2,test_number_of_words)

----------------------------------------------------------------------------------------------------
Timestep 0/3
----------------------------------------------------------------------------------------------------


	Done 0/2
	Done 1/2
[0.03952746930657039, 0.04122628159712735]

Fitting time : 0.33s

----------------------------------------------------------------------------------------------------
Timestep 1/3
----------------------------------------------------------------------------------------------------


	Done 0/2
	Done 1/2
[0.0527796056484553, 0.06979573092922156]

Fitting time : 0.33s

----------------------------------------------------------------------------------------------------
Timestep 2/3
----------------------------------------------------------------------------------------------------


	Done 0/2
	Done 1/2
[0.08259622498258, 0.07429565904991407]

Fitting time : 0.37s

Total fitting time for 3 timestep & 30 number of words is : 1.03s


In [96]:
print('Final Topic Coherence : ')
print(test_2_)

Final Topic Coherence : 
[[0.03952747 0.04122628]
 [0.05277961 0.06979573]
 [0.08259622 0.07429566]]


In [97]:
print('TC with method 1 :')
print(test_1_)
print('\n\tFitting time : 130.15s')
print('\n')
print('TC with method 2 :')
print(test_2_)
print('\n\tFitting time : 4.31s for creating one-hot-BOW, 1.03s for fitting time')

TC with method 1 :
[[0.03952747 0.04122628]
 [0.05277961 0.06979573]
 [0.08259622 0.07429566]]

	Fitting time : 130.15s


TC with method 2 :
[[0.03952747 0.04122628]
 [0.05277961 0.06979573]
 [0.08259622 0.07429566]]

	Fitting time : 4.31s for creating one-hot-BOW, 1.03s for fitting time


The method we implemented seems clearly better, as it takes 130 times less computational time to compute for 30 words. As the number of words increases, the advantage of our technic is exploding